In [77]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

---

## 1. Price parsing

- `original_price`: Giá gốc (giá chưa giảm, đơn vị: VND)
- `sale_price`: Giá khuyến mãi (giá đã giảm, đơn vị: VND)
- `precent_discount`: Phần trăm số tiền đã giảm (đơn vị: %)

<pre>
- Có giá khuyến mãi: 
    &lt;fare-sale&gt; chứa giá khuyến mãi
        &lt;fareSmall&gt; chứa giá gốc và phần trăm ưu đãi
- Không có giá khuyến mãi:
    &lt;fare&gt; chứa giá gốc
        &lt;fareSmall&gt; không chứa dữ liệu
</pre>

In [78]:
def has_no_discount_price(block) -> bool:
    '''
    Phân loại chuyến xe này có giá khuyến mãi hay không \n
    True: không có giá khuyến mãi \n
    False: có giá khuyến mãi
    '''
    fare = block.find('div', class_='fare')      
    
    if fare: 
        return True  # khong co gia khuyen mai
    
    return False     # co gia khuyen mai

def parse_fare(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi không có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi = None, phần trăm khuyến mãi = None
    '''

    sale_price = None
    percent_discount = None

    fare = block.find('div', class_='fare')
    original_price = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()    
    
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

def parse_fare_small(block):
    '''
    Dành cho trường hợp có khuyến mãi \n
    Trả về: giá gốc, phần trăm khuyến mãi
    '''

    fare_small = block.find('div', class_='fareSmall')

    original_price = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip() if fare_small else None
    try:
        if fare_small.find('div', class_='percent'):
            percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
        else:
            percent_discount = None
    except Exception:
        percent_discount = None
        
    return original_price, percent_discount

def parse_fare_sale(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi, phần trăm khuyến mãi
    '''
    

    fare_sale = block.find('div', class_='fare-sale')
    original_price, percent_discount = parse_fare_small(block)
    sale_price = None
    if fare_sale and fare_sale.get_text(strip=True):
        sale_price = fare_sale.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
        
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

In [79]:
def parse_price(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe \n
    Trả về: giá gốc, giá khuyến mãi (nếu có), phần trăm khuyến mãi (nếu có)
    '''
    
    if has_no_discount_price(block):
        return parse_fare(block)
    else:
        return parse_fare_sale(block)

---

## 2. Bus info parsing

In [80]:
def parser_trip_bus_info(container):
    '''
    Trích xuất thông tin từ một container chứa thông tin chuyến đi. \n
    Trả về Tuple: tên nhà xe, đánh giá nhà xe, loại ghế.
    '''
    
    # bus name / company name
    bus_element = container.find('div', class_='bus-name')
    bus_name = bus_element.get_text(strip=True) if bus_element else None

    # bus rating
    rating_element = container.find('div', class_='bus-rating').find('span')
    bus_rating = rating_element.get_text(strip=True) if rating_element else None

    # seat_type
    seat_type = container.find('div', class_='seat-type')
    seat_type = seat_type.get_text(strip=True) if seat_type else None

    return {
        'bus_name': bus_name,
        'bus_rating': bus_rating,
        'seat_type': seat_type
    }

---

## 3. Route parsing

In [81]:
# Dữ liệu này nằm ở ô filter chuyến đi

def parse_route_info(block):
    '''
    Trích xuất dữ liệu từ filter của trang web \n
    Trả về: ngày khởi hành, nơi xuất phát (thành phố hiện tại), nơi đến (nơi đặt vé đến)
    '''

    departure_date, start_point, destination = None, None, None

    try:
        departure_date = block.find('p', class_='date-input-value').get_text(strip=True)
        start_point = block.find(id="from_input").get('value')
        destination = block.find(id="to_input").get('value')
    except Exception:
        pass

    return {
        'departure_date': departure_date,
        'start_point': start_point,
        'destination': destination
    }

---

## 4. Details trip info

### 4.1 Departure

In [82]:
# Dữ liệu này nằm trong container > 'from_content'

def parse_departure_trip_info(from_content):
    """
    Trích xuất thông tin điểm đi từ một container 'from_content'.
    Trả về một tuple chứa: giờ khởi hành, địa điểm đón khách.
    """
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not from_content:
        return None, None

    # departure time
    from_hour_tag = from_content.find('div', class_='hour')
    departure_time = from_hour_tag.get_text(strip=True) if from_hour_tag else None

    # departure place
    from_place_tag = from_content.find('div', class_='place')
    pick_up_point = from_place_tag.get_text(strip=True) if from_place_tag else None
    
    return {
        'departure_time': departure_time,
        'pick_up_point': pick_up_point
    }

### 4.2 Arrival

In [83]:
def parse_arrival_trip_info(to_content):
    """
    Trích xuất thông tin điểm đến từ một container 'to_content'.\n
    Trả về một tuple chứa: ngày đến, thời gian đến, điểm trả khách.
    """
    
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not to_content:
        return None, None, None

    # lấy ngày đến
    date_arrival_tag = to_content.find('span', class_="text-date-arrival-time")
    arrival_date = date_arrival_tag.get_text(strip=True) if date_arrival_tag else None
    
    
    # lấy giờ và địa điểm trả khách
    content_to_info = to_content.find('div', class_='content-to-info')
    if content_to_info:
        to_hour_tag = content_to_info.find('div', class_='hour')
        arrival_time = to_hour_tag.get_text(strip=True) if to_hour_tag else None
        
        to_place_tag = content_to_info.find('div', class_='place')
        drop_off_point = to_place_tag.get_text(strip=True) if to_place_tag else None
        
    return {
        'arrival_date': arrival_date,
        'arrival_time': arrival_time,
        'drop_of_point': drop_off_point
    }


---

In [84]:
def get_departure_arrival_trip(container):
    """
    Trích xuất thông tin chi tiết về chuyến đi (giờ, nơi đi - đến, thời gian di chuyển). \n
    Trả về: dict chứa thông tin khởi hành, điểm đến và thời lượng chuyến.
    """
    
    # Tìm khối chứa thông tin đi và đến
    from_to_content = container.find('div', class_="from-to-content")

    # Nếu không tìm thấy, trả về dict rỗng có cấu trúc sẵn
    if not from_to_content:
        return {
            "duration": None,
            "from_hour": None,
            "from_place": None,
            "departure_date": None,
            "arrival_date": None,
            "to_hour": None,
            "to_place": None,
        }

    # Lấy thông tin nơi khởi hành
    from_content = from_to_content.find('div', class_='content from')
    dict_departure_info = parse_departure_trip_info(from_content)

    # Lấy thông tin nơi đến
    to_content = from_to_content.find('div', class_='content to')
    dict_arrival_info = parse_arrival_trip_info(to_content)

    # Lấy thời gian di chuyển
    duration_tag = from_to_content.find('div', class_="duration")
    duration = duration_tag.get_text(strip=True) if duration_tag else None

    # Gộp toàn bộ thông tin lại
    trip_data = dict_departure_info | dict_arrival_info | {'duration': duration}
    
    return trip_data


In [85]:
def gather_trip_info(block):
    '''
    Tập hợp toàn bộ thông tin của 1 chuyến xe từ 1 khối dữ liệu (block). \n
    Trả về: dict chứa thông tin xe, lịch trình và giá vé.
    '''
    
    # Lấy thông tin chính của nhà xe
    dict_bus_info = parser_trip_bus_info(block)
    
    # Lấy thông tin giờ đi - giờ đến, điểm đón - trả
    dict_trip_details = get_departure_arrival_trip(block)
    
    # Lấy thông tin giá vé (giá gốc, giá khuyến mãi)
    dict_price = parse_price(block)
    
    # Gộp tất cả dữ liệu vào 1 dictionary duy nhất
    trip_data = dict_bus_info | dict_trip_details | dict_price
    
    return trip_data


---

rating

In [86]:
def extract_rating_from_container(container):
    '''
    Trích xuất thông tin đánh giá (rating) của từng nhà xe trong 1 container. \n
    Trả về: list các cặp (rate_title, rate_point) hoặc [(None, None)] nếu không có dữ liệu.
    '''
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')   # Tìm tất cả khối chứa thông tin đánh giá
        
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')   # Mỗi phần tử chứa tiêu đề và điểm
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)   # Tiêu đề đánh giá
                rate_point = rate_ps[1].get_text(strip=True)   # Điểm đánh giá
                ratings.append((rate_title, rate_point))       # Lưu vào danh sách
        
        if ratings:
            return ratings     # Trả về danh sách nếu có dữ liệu
        else:
            return [(None, None)]   # Không có dữ liệu đánh giá
            
    except Exception:
        return [(None, None)]   # Trường hợp lỗi vẫn trả về giá trị mặc định


total

In [87]:
# Gộp dữ liệu chuyến xe và các đánh giá lại với nhau
def get_all_bus_trip_info(soup):
    '''
    Trích xuất và gộp thông tin chuyến xe, tuyến đường và đánh giá nhà xe thành một DataFrame duy nhất. \n
    Trả về: DataFrame chứa toàn bộ dữ liệu chuyến xe.
    '''

    dict_route = parse_route_info(soup)   # Lấy thông tin tuyến đường (đi - đến)
    containers = soup.find_all("div", class_="container")   # Tìm tất cả container chứa chuyến xe

    lst_trips_info = []

    for container in containers:
        # Lấy thông tin chuyến xe và gộp với tuyến đường
        dict_trip_info = gather_trip_info(container) | dict_route   
        df_trip_info = pd.DataFrame([dict_trip_info])  # Đưa về 1 hàng dataframe

        # Lấy dữ liệu đánh giá của nhà xe
        lst_detail_rating = extract_rating_from_container(container)  
        df_ratings = pd.DataFrame([dict(lst_detail_rating)])  # Đưa list -> dict -> df 1 hàng

        # Gộp 2 dataframe lại (thông tin + đánh giá)
        df = pd.concat([df_trip_info, df_ratings], axis=1)

        lst_trips_info.append(df)

    # Gộp toàn bộ chuyến xe lại thành 1 dataframe
    all_trips_info = pd.concat(lst_trips_info, ignore_index=True)

    return all_trips_info

---

In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## 5. Handle Button logic

### 5.1. Button `Xem thêm chuyến`

In [89]:
def click_load_more(driver):
    """
    Tự động nhấn nút 'Xem thêm chuyến' nhiều lần cho đến khi không còn nút nào hiển thị.
    Dừng lại khi trang đã tải hết tất cả chuyến xe.
    """
    
    while True:
        try:
            load_more_span = driver.find_element(By.XPATH, "//span[text()='Xem thêm chuyến']")
            load_more_button = load_more_span.find_element(By.XPATH, "./ancestor::button")
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            time.sleep(random.uniform(2, 3))
        except NoSuchElementException:
            break
        except ElementClickInterceptedException:
            time.sleep(2)

### 5.2. Button `Xem các đánh giá`

In [90]:
# def open_all_ratings(driver, max_wait: int = 15):
#     """
#     Mở tất cả phần đánh giá (rating) của các chuyến xe sau khi đã load toàn bộ danh sách.

#     Parameters
#     ----------
#     driver : webdriver.Chrome
#         Đối tượng điều khiển trình duyệt Selenium.
#     max_wait : int
#         Thời gian tối đa (giây) để chờ các phần tử rating xuất hiện.

#     Returns
#     -------
#     bool
#         True nếu đã click được ít nhất 1 rating, False nếu không có phần rating nào.
#     """
    
#     try:
#         # Chờ các icon ngôi sao xuất hiện
#         stars = WebDriverWait(driver, max_wait).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-rating-button .anticon-star"))
#         )

#         if not stars:
#             print("⚠️  Không tìm thấy phần đánh giá nào trong trang — bỏ qua bước click.")
#             return False

#         print(f"✅ Tìm thấy {len(stars)} phần rating — tiến hành click mở chi tiết...")

#         for star in stars:
#             try:
#                 driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
#                 time.sleep(1.5)
#                 ActionChains(driver).move_to_element(star).click().perform()
#                 time.sleep(1)
#             except Exception as click_err:
#                 print(f"❌ Không thể click rating: {click_err}")

#         print("✅ Đã mở tất cả phần đánh giá.")
#         return True

#     except Exception as e:
#         print("⚠️  Không thể load hoặc click rating:", e)
#         return False


def open_all_ratings(driver, max_clicks=30, delay_range=(0.5, 1.2)):
    """
    Mở tất cả các cửa sổ rating trên trang Vexere (nếu có).
    - Giới hạn số lượt click để tránh treo.
    - Tự bỏ qua khi element không khả dụng.
    """

    wait = WebDriverWait(driver, 10)

    try:
        # Lấy toàn bộ các icon ngôi sao hiện có
        stars = wait.until(
            EC.presence_of_all_elements_located(
                (By.CLASS_NAME, "bus-rating-button")
            )
        )
        print(f"⭐ Tìm thấy {len(stars)} icon rating.")

        clicked = 0

        for star in stars:
            if clicked >= max_clicks:
                print(f"⏹ Dừng lại sau {max_clicks} lần click (tránh treo).")
                break

            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
                time.sleep(random.uniform(*delay_range))

                # Dùng click an toàn bằng JS (ít lỗi hơn ActionChains)
                driver.execute_script("arguments[0].click();", star)
                clicked += 1

                print(f"✅ Click rating {clicked}/{len(stars)}")
                time.sleep(random.uniform(*delay_range))

            except Exception as e:
                print("⚠️ Bỏ qua 1 icon do lỗi:", e)
                continue

        print("🎯 Hoàn tất mở rating!")

    except Exception as e:
        print("❌ Không thể lấy danh sách rating:", e)

### 5.3. Button `Tìm kiếm`

In [91]:
def click_search_button(driver):  
    """
    Click vào nút tìm kiếm trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.

    Returns
    -------
    bool
        True nếu click thành công, False nếu xảy ra lỗi.
    """
    
    try:
        button = driver.find_element(By.CLASS_NAME,"button-search")
        button.click()
        return True
    except Exception:
        return False

## 6. Automate the process of filtering website data

In [92]:
def target_time(days=0):
    """
    Trả về ngày và tháng-năm mục tiêu cách hiện tại `days` ngày.

    Parameters
    ----------
    days : int, optional
        Số ngày cộng thêm từ ngày hiện tại (mặc định = 0).

    Returns
    -------
    dict
        {'day': '15', 'month_year': '10-2025'}
    """
    target_date = datetime.today() + timedelta(days=days)  # Ngày mục tiêu = ngày hiện tại + khoảng thời gian sau k days ngày
    month_id = f"{target_date.month:02d}-{target_date.year}"
    day = str(target_date.day)
    return {
        'day':day,
        'month_year':month_id
    }

In [93]:
def filter_box(driver,start_city:str, destination_city:str, days=0):
    """
    Chọn điểm đi, điểm đến và ngày khởi hành trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.
    start_city : str
        Tên thành phố khởi hành.
    destination_city : str
        Tên thành phố điểm đến.
    days : int, optional
        Số ngày tính từ hôm nay để chọn ngày đi (mặc định = 0).

    Returns
    -------
    bool
        True nếu chọn ngày thành công, False nếu xảy ra lỗi.
    """

    # Tìm ô chứa nơi khởi hành và nơi đến
    box_department = driver.find_element(By.ID, 'from_input')
    box_arrival = driver.find_element(By.ID, 'to_input')

    # Nhập dữ liệu
    box_department.send_keys(start_city)
    box_arrival.send_keys(destination_city)
    
    # click vào chọn ngày đi để hiển thị các lựa chọn
    driver.find_element(By.CLASS_NAME, "departure-date-select").click() 
    time.sleep(1)

    # Gọi hàm target_time() -> ngày, tháng mình muốn hiển thị từ trang web
    target_day, target_month = target_time(days).values() 

    # Chọn ngày trong month_section
    month_section = driver.find_element(By.ID, target_month)
    day_elements = month_section.find_elements(By.CSS_SELECTOR, "p.day")
    
    for day in day_elements:
        if day.text == target_day:
            try:
                day.click()
                break
            except Exception:
                return False
            
    return True

### 7. Select arrival place

In [94]:
import pandas as pd
df = pd.read_csv('../../data/raw/routes.csv')
df

,start_point,destination_1,destination_2,destination_3,destination_4,destination_5,destination_6,destination_7,destination_8,destination_9,destination_10
0,Hà Nội,Hải Phòng,Nghệ An,Sơn La,Hà Giang,Quảng Ninh,Thanh Hóa,SaPa,Ninh Bình,NaN,NaN
1,Quảng Ninh,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
2,Ninh Bình,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
3,Đà Nẵng,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
4,Sài Gòn,Bình Thuận,Ninh Thuận,Đắk Lắk,Gia Lai,Phú Yên,Nha Trang,Bà Rịa - Vũng Tàu,NaN,NaN,NaN
5,Sa Pa,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
6,Vũng Tàu,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
7,Đà Lạt,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
8,Nha Trang,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
9,Phan Thiết,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết


# FLOW OFFICIAL

---

### check data trong database

In [95]:
# def open_all_ratings(driver, max_wait: int = 15, db_connection=None):
#     """
#     Mở các đánh giá chưa có trong database.

#     Parameters
#     ----------
#     driver : webdriver.Chrome
#         Đối tượng điều khiển trình duyệt Selenium.
#     max_wait : int
#         Thời gian tối đa (giây) để chờ các phần tử rating xuất hiện.
#     db_connection : database connection object
#         Kết nối đến database để kiểm tra dữ liệu

#     Returns
#     -------
#     bool
#         True nếu đã click được ít nhất 1 rating, False nếu không có phần rating nào.
#     """
#     try:
#         # Chờ các container chứa thông tin chuyến xe xuất hiện
#         containers = WebDriverWait(driver, max_wait).until(
#             EC.presence_of_all_elements_located((By.CLASS_NAME, "container"))
#         )

#         if not containers:
#             print("⚠️  Không tìm thấy chuyến xe nào trong trang.")
#             return False

#         for container in containers:
#             try:
#                 # Lấy thông tin nhận dạng chuyến xe (ví dụ: tên nhà xe, giờ khởi hành)
#                 bus_name = container.find_element(By.CLASS_NAME, "bus-name").text
#                 departure_time = container.find_element(By.CLASS_NAME, "hour").text
                
#                 # Kiểm tra trong database
#                 if db_connection:
#                     cursor = db_connection.cursor()
#                     cursor.execute("""
#                         SELECT id FROM bus_trips 
#                         WHERE bus_name = %s AND departure_time = %s
#                     """, (bus_name, departure_time))
                    
#                     if cursor.fetchone():
#                         print(f"Skip rating for {bus_name} - {departure_time} (already in DB)")
#                         continue

#                 # Nếu chưa có trong DB, mở rating
#                 star = container.find_element(By.CSS_SELECTOR, ".bus-rating-button .anticon-star")
#                 driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
#                 time.sleep(1.5)
#                 ActionChains(driver).move_to_element(star).click().perform()
#                 time.sleep(1)
                
#             except Exception as click_err:
#                 print(f"❌ Lỗi xử lý container: {click_err}")
#                 continue

#         print("✅ Đã xử lý tất cả các đánh giá.")
#         return True

#     except Exception as e:
#         print("⚠️  Lỗi chung:", e)
#         return False

## Crawl rating data

# -- Main --

In [96]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

flow: filter_box(driver, 'Sài Gòn/Hà Nội', list, days=for i in range) -> click_search_button(driver)

In [97]:

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.get("https://vexere.com/vi-VN/ve-xe-khach-tu-ha-noi-di-thanh-hoa-124t1561.html?date=15-10-2025&v=8&nation=84")

# # Click 'xem thêm chuyến' đến hết -> hiển thị tất cả các chuyến xe
# click_load_more(driver)


# open_all_ratings(driver)
# time.sleep(1.5)
# soup = BeautifulSoup(driver.page_source, "html.parser")

# lst_trips_info = []
# df_trips_info = get_all_bus_trip_info(soup)
# with open('hn_th_15102025.html', 'w', encoding='utf-8') as f:
#     f.write(soup.prettify())

# df_trips_info.to_csv("../../data/raw/hn_th_15102025.csv", index=False)
# driver.quit()


In [98]:
arrivals_HaNoi = ['Hải Phòng','Nghệ An','Sơn La','Hà Giang','Quảng Ninh','Thanh Hóa'',SaPa','Ninh Bình']
arrivals_SaiGon = ['Gia Lai','Bình Thuận','Ninh Thuận','Đắk Lắk','Phú Yên','Nha Trang','Bà Rịa - Vũng Tàu']

In [99]:
URL = 'https://vexere.com/'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(URL)

In [ ]:
departure_place = 'Sài Gòn' # Nơi xuất phát
days = 1

day, month_year = target_time(days).values()
month_year = day.replace('-', '_')

for i in arrivals_SaiGon:
    arrival_place = i

    
    
    filter_sucess = filter_box(driver, start_city=departure_place, destination_city=arrival_place, days=days)

    if filter_sucess:
        search_suess = click_search_button(driver)
    
    click_load_more(driver)

    if search_suess:
        open_all_ratings(driver)

    time.sleep(1.5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    lst_trips_info = []

    try:
        df_trips_info = get_all_bus_trip_info(soup)
    except Exception:

        with open(f'../../data/site/{departure_place}_{arrival_place}_{day}_{month_year}.html', 'w', encoding='utf-8') as f:
            f.write(soup.prettify())
        
        print("Đã lưu thành .html")

    df_trips_info.to_csv(f"../../data/raw/{departure_place}_{arrival_place}_{day}_{month_year}.csv", index=False)
    print("Lấy dữ liệu .csv thành công")

    driver.quit()
    break


⭐ Tìm thấy 20 icon rating.
✅ Click rating 1/20
✅ Click rating 2/20
✅ Click rating 3/20
✅ Click rating 4/20
✅ Click rating 5/20
✅ Click rating 6/20
✅ Click rating 7/20
✅ Click rating 8/20
✅ Click rating 9/20
✅ Click rating 10/20
✅ Click rating 11/20
✅ Click rating 12/20
✅ Click rating 13/20
✅ Click rating 14/20
✅ Click rating 15/20
✅ Click rating 16/20
✅ Click rating 17/20
✅ Click rating 18/20
✅ Click rating 19/20
✅ Click rating 20/20
🎯 Hoàn tất mở rating!
Lấy dữ liệu .csv thành công


In [101]:
df_trips_info

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_date,arrival_time,drop_of_point,duration,original_price,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Hoàng Thuỷ,4.7 (3363),Limousine 34 Giường VIP,18:45,• Bến xe Miền Đông - Quầy vé 37,(18/10),05:45,• Bến Xe Đức Long Gia Lai,11h,350.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.8,4.7,4.6,4.6,4.6,4.6,4.9
1,Việt Tân Phát,4.6 (92),34 Phòng,17:45,• Bến xe Miền Đông,(18/10),04:55,• Bến xe Đắk Đoa,11h10m,370.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.7,4.7,4.7,4.5,4.6,4.5,4.6
2,Sinh Diên Hồng,4.6 (227),Limousine 34 chỗ,17:30,• Bến xe An Sương,(18/10),04:15,• Bến Xe Đức Long Gia Lai,10h45m,370.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.6,4.6,4.8,4.7,4.6,4.5,4.8
3,Kính Diên Hồng,4.6 (1127),Limousine giường nằm 34 chỗ,18:40,• Bến Xe Miền Đông - Quầy vé 19,(18/10),05:40,• Bến Xe Đức Long Gia Lai,11h,350.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.7,4.6,4.7,4.6,4.5,4.5,4.5
4,An Phát,4.5 (375),Limosuine 24 Phòng,17:30,• BX Miền Đông - Cổng 4,(18/10),07:15,• Bến Xe K-Bang,13h45m,550.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.6,4.5,4.6,4.5,4.3,4.5,4.8
5,Vương Tấn Dũng,4.8 (372),Limousine 24 Phòng,17:45,• VP QL13,(18/10),06:00,• Bến xe An Khê,12h15m,500.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.8,4.7,4.8,4.8,4.8,4.8,4.8
6,Đức Đạt,4.7 (974),Limousine giường nằm 34 chỗ,18:25,• Bến xe Miền Đông - Quầy vé 17,(18/10),04:50,• Bến Xe Đức Long Gia Lai,10h25m,350.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.6,4.5,4.5,4.5,4.4,4.4,4.8
7,Thuận Tiến,4.7 (4930),Limousine 34 giường,18:30,• Bến Xe Miền Đông - Quầy vé 24,(18/10),06:55,• Bến Xe Đức Long Gia Lai,12h25m,350.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.7,4.7,4.8,4.6,4.6,4.6,4.8
8,Tấn Hưng (Gia Lai),4.4 (101),Limousine 24 Phòng,17:30,• Bến Xe Miền Đông - Phòng vé số 77,(18/10),07:55,• Bến xe khách Kbang,14h25m,650.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.5,4.2,4.4,4.5,4.4,4.4,3.6
9,Bảy Lang,4.7 (271),Limousine 34 giường,18:30,"• Bến xe miền đông cũ (Dãy số 6, Ô C14)",(18/10),05:30,• Bến Xe Đức Long Gia Lai,11h,350.000,...,"T6, 17/10/2025",Sài Gòn,Gia Lai,4.8,4.6,4.6,4.7,4.5,4.5,4.4
